In [3]:
import pandas as pd

# 1. Dateien laden
df_all = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx", engine="openpyxl")
df_clustered = pd.read_excel("../../data/df_de_final_clustered_bert_noise_reclustered.xlsx", engine="openpyxl")

# 2. Mapping: Duplikat_Gruppe → Cluster_HDBSCAN
mapping_cluster = df_clustered.set_index("Duplikat_Gruppe")["Cluster_HDBSCAN"].to_dict()

# 3. Cluster_HDBSCAN ins vollständige DataFrame übertragen
df_all["Cluster_HDBSCAN"] = df_all["Duplikat_Gruppe"].map(mapping_cluster).fillna(-1).astype(int)


In [4]:
# 4. Mapping: Duplikat_Gruppe → noise_cluster_cleaned (nur für ursprüngliches Noise)
mapping_noise = df_clustered.set_index("Duplikat_Gruppe")["noise_cluster_cleaned"].to_dict()

# 5. Nur auf Fragen anwenden, die Cluster_HDBSCAN == -1
df_all["noise_cluster_cleaned"] = df_all.apply(
    lambda row: mapping_noise.get(row["Duplikat_Gruppe"], -99) if row["Cluster_HDBSCAN"] == -1 else -99,
    axis=1
)


In [5]:
df_all.to_excel("../../data/df_de_final_clustered_all_mapped.xlsx", index=False, engine="openpyxl")
print("Datei gespeichert: df_de_final_clustered_all_mapped.xlsx")


Datei gespeichert: df_de_final_clustered_all_mapped.xlsx


In [6]:
# Clustergrössen berechnen (nur für echte Cluster)
cluster_sizes = df_all[df_all["Cluster_HDBSCAN"] != -1]["Cluster_HDBSCAN"].value_counts()
df_all["Cluster_Grösse"] = df_all["Cluster_HDBSCAN"].map(cluster_sizes)


In [13]:
df_all["is_clustered"] = df_all["Cluster_HDBSCAN"] != -1

df_all_sorted = df_all.sort_values(
    by=[
        "is_clustered",             # True (echte Cluster) zuerst
        "Cluster_Grösse",           # grössere Cluster zuerst
        "Cluster_HDBSCAN",          # Cluster-ID
        "noise_cluster_cleaned",    # innerhalb Noise: Subcluster
        "Frage_Text"                # alphabetisch
    ],
    ascending=[False, False, True, True, True]
).reset_index(drop=True)


In [17]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import PatternFill

# === 5. Excel farbig exportieren ===
wb = Workbook()
ws = wb.active
ws.title = "Finale Clusteransicht"

# Farben definieren
trennung_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")  # hellblau
variante_fill = PatternFill(start_color="FFFACD", end_color="FFFACD", fill_type="solid")  # hellgelb
noise_colors = ["E0F7FA", "FFF3E0", "E8F5E9", "F3E5F5", "FCE4EC", "F1F8E9", "F9FBE7", "EDE7F6"]

# Header schreiben
ws.append(df_all_sorted.columns.tolist())

# Initialwerte für Vergleiche
prev_cluster = None
prev_duplikat_gruppe = None
noise_color_map = {}
color_index = 0

# Zeilen schreiben
for idx, row in df_all_sorted.iterrows():
    cluster_id = row["Cluster_HDBSCAN"]
    duplikat_gruppe = row["Duplikat_Gruppe"]
    noise_id = row["noise_cluster_cleaned"]

    # Trennzeile bei neuem Cluster
    if cluster_id != prev_cluster and prev_cluster is not None:
        ws.append(["" for _ in range(len(df_all_sorted.columns))])
        for cell in ws[ws.max_row]:
            cell.fill = trennung_fill

    # Zeile einfügen
    ws.append(row.tolist())

    # Zellfarben anwenden
    for col_idx, cell in enumerate(ws[ws.max_row]):
        if cluster_id != -1:
            # Varianten im Cluster hervorheben
            if duplikat_gruppe != prev_duplikat_gruppe:
                cell.fill = variante_fill
        else:
            # Farben für Subcluster der Noise-Gruppe
            if noise_id not in noise_color_map:
                noise_color_map[noise_id] = PatternFill(
                    start_color=noise_colors[color_index % len(noise_colors)],
                    end_color=noise_colors[color_index % len(noise_colors)],
                    fill_type="solid"
                )
                color_index += 1
            cell.fill = noise_color_map[noise_id]

    # Merker aktualisieren
    prev_cluster = cluster_id
    prev_duplikat_gruppe = duplikat_gruppe

# === 6. Datei speichern ===
output_path = "../../data/df_de_final_clustered_all_colored.xlsx"
wb.save(output_path)
print(f"Datei erfolgreich exportiert: {output_path}")


Datei erfolgreich exportiert: df_de_final_clustered_all_colored.xlsx
